## 初步验证

### 1. CoT-任务拆解能力

In [2]:
import time
import pandas as pd
from IPython.display import HTML
from tools.utils import get_features,disp,highlight_differences

from tools.inferences import openai_inference,transformers_inference
from prompts import cot_prompt as prompt

# pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', None)

top_p = 0.7
temperature = 0.6
model_uid = 'glm-4-9b-chat'


In [3]:
if input('更新prompt重跑？(y/n)') == 'y':
    dev_df = pd.read_excel('dev.xlsx')
    dev_df['response'] = ''
    start_time = time.time()
    for index, row in dev_df.iterrows():
        query = prompt.format(row['query'])
        response = openai_inference(query, model_uid, top_p, temperature)
        if index % 100 == 0:
            passed_time = time.time() - start_time
            # print(f'============{index}, passed time: {passed_time:.2f}s=========== \n')
            # print(f'response: {response}')
        dev_df.loc[index, 'response'] = response
        # 在csv中追加行
        with open('dev_response.log', 'a') as f:
            f.write(f'query:{row["query"]}\nresponse:{response}\n\n')
        
    dev_df.to_excel('dev_response.xlsx', index=False)
else:
    dev_df = pd.read_excel('dev_response.xlsx')

In [4]:
# 解析label中的 api链
dev_df = pd.concat([dev_df,dev_df.label.apply(get_features,args=('A',))],axis=1)

# 解析response中的 api链
dev_df['B_api_list'] = ''
for i,row in dev_df.iterrows():
    response = row['response']
    try:
        b_api_list = eval(response[response.rfind('api_list:')+9:].strip())
        b_api_list = [(x[0],'查询'+x[1])  if x[0] in ['基金查询','条件选基','股票查询','条件选股'] else x for x in b_api_list]
    except:
        print('Error: row',i,'response:',response)
        b_api_list = []
    dev_df.loc[i,'B_api_list'] = '<br>'.join([str(x) for x in b_api_list])
    
dev_df[['A_api_list','B_api_list']] = dev_df.apply(highlight_differences, args=('A_api_list','B_api_list'), axis=1)

['api_1的结果', 'api_2的结果', {'api_3的结果 方向': '流入', 'api_4的结果 方向': '流出'}]
Error: row 628 response: 
思考过程：
1. 首先需要确定“数据源网络IO异常”对应的股票代码。
2. 然后根据股票代码查询上个月的最高价和最低价。

结果：
```json
api_list: [
    ("股票查询", "代码"),
    ("股票查询", "最高价"),
    ("股票查询", "最低价")
]
```


In [5]:
# 准确率
(dev_df.A_api_list == dev_df.B_api_list).sum()/dev_df.shape[0]

np.float64(0.3626373626373626)

In [6]:
# 可视化
disp(dev_df[['query','A_api_list','B_api_list']],cols_width=[200,300,300])

,query,A_api_list,B_api_list
0,我想知道国金核心资产A的基金经理是谁，以及他的年化回报率和管理的总规模,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')('基金查询', '查询基金经理管理规模')"
1,我想了解汇丰晋信2016C的基金经理是谁，目前的基金规模以及申购费率是多少?,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询规模')('基金查询', '查询申购费率')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金规模')('基金查询', '查询申购费率')"
2,我现在要买1000份兴业聚惠A基金，我需要支付多少申购费？,"('基金查询', '查询代码')('基金查询', '查询申购费率')('基金查询', '查询单位净值')('数值计算', '乘法计算')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询申购费率')('基金查询', '查询单位净值')('数值计算', '乘法计算')('数值计算', '乘法计算')('数值计算', '乘法计算')"
3,南方新能源C的基金经理是谁，他管理的基金总规模有多大？,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金规模')"
4,汇添富添添鑫A基金近1年的最大回撤及其同类排名信息是怎样的？,"('基金查询', '查询代码')('基金查询', '查询近期最大回撤')('基金查询', '查询近期最大回撤同类排名')","('基金查询', '查询代码')('基金查询', '查询近期最大回撤')('基金查询', '查询近期最大回撤同类排名')"
5,昨天成交量是70万8300股，而上个月最低价是5.71元的股票有哪些？,"('条件选股', '查询成交量')('条件选股', '查询最低价')('逻辑运算', '与运算')","('条件选股', '查询成交量')('条件选股', '查询最低价')('逻辑运算', '与运算')"
6,我计划用10000块钱购买金鹰稳进配置C，我需要支付多少的申购费用,"('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')"
7,我5000块要全投鹏华一带一路(LOF)A，扣除认购费用后真正可以投入的金额是多少,"('基金查询', '查询代码')('基金查询', '查询认购费率')('数值计算', '乘法计算')('数值计算', '减法计算')","('基金查询', '查询代码')('基金查询', '查询认购费率')('数值计算', '乘法计算')('数值计算', '减法计算')"
8,我如果一年前用2000块买国金惠安C，现在的基金总价值是多少,"('基金查询', '查询代码')('基金查询', '查询近期收益率')('数值计算', '乘法计算')('数值计算', '加法计算')","('基金查询', '查询代码')('基金查询', '查询单位净值')('数值计算', '除法计算')('数值计算', '乘法计算')"
9,你知道有哪些基金在最近一个月的最大回撤在它们同类中的排名是第3654位，而且在过去五年里它们的最大回撤是36.17%吗？,"('条件选基', '查询近期最大回撤同类排名')('条件选基', '查询近期最大回撤')('逻辑运算', '与运算')","('条件选基', '查询近期最大回撤同类排名')('条件选基', '查询近期最大回撤')('逻辑运算', '与运算')"


In [8]:
# 在res字符串中，找到最后一个"结果" 之后的内容，即为答案
res = res[res.rfind("结果"):]

In [9]:
res

'结果\n    api_list: [\n        ("股票查询", "代码", "三羊马"),\n        ("股票查询", "最高价", "三羊马"),\n        ("通用运算符", "除法计算", 1000000, ["股票查询", "最高价", "三羊马"])\n    ]'

---

### 2. 纠错能力

In [ ]:
错误类型

# 规则验证？ 
    # 行业知识引入
        如：股票、基金的买卖，都涉及哪些费用
    # 基本
        价格查询：
    

In [ ]:
prompt = '''
    领域知识：
        。。

'''